# Scraping Toronto postal codes from wiki page
## The first notebook

In [3]:
wiki_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # wiki page url
wiki_url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
# modules importing
import pandas as pd
import requests
from bs4 import BeautifulSoup

We will create an empty DataFrame with three columns: _PostalCodes_, _Borough_, _Neighborhood_:

In [60]:
# creating DataFrame
columns = ['PostalCodes',
          'Borough',
          'Neighborhood']
df = pd.DataFrame(columns=columns)

Next cell will perform data extracting and cleaning:

In [59]:
# wiki table extracting
results = requests.get(wiki_url).text
table = BeautifulSoup(results).find("table") # extracting the table from a wiki page
rows = table.find_all("tr")
raw_data = [] # list of dictionaries in order to build DataFrame
for row in rows: 
    for cell in row.find_all("td"):
        postal_code = cell.p.b.text
        try:
            borough, neighborhood = cell.span.text.split('(')[:2]
        except Exception as err:
            continue
        neighborhood = neighborhood.strip(')').replace(' /', ',').replace(')',' ').strip(' ')
        borough = borough.strip(' ')
        raw_data.append({'PostalCodes': postal_code,
                   'Borough': borough,
                   'Neighborhood': neighborhood})

df = pd.DataFrame(raw_data)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(11)

,PostalCodes,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [43]:
df.shape

(101, 3)